# Segundo Parcial: Resolución de una Caldera Humotubular
En esta instancia se resolverá en forma de un programa el calculo de los balances y las ecuaciones de transferencia de calor de una caldera humotubular para unos datos especificados por el usuario que son:
- Analisis último del combustible en fracción másica o molecular, dado en base seca
- Agua en combustible, dato dado en base seca
- Datos de la combustión (Exceso-CO, Exceso-CO2, CO2-CO)
- Poder calorífico superior (Qps)
- Presión de trabajo de la caldera (p_trabajo)
- Kg de vapor por hora (Gv_t)
- Temperatura de entrada del agua (t_agua)
- Largo y diámetro del hogar (o el área de transferencia).
- Cantidad de pasos por tubos
- Cantidad, largo y diámetro de cada paso de tubos.
<br>
Con estos datos deberemos encontrar:
- Composición de humos en base seca
- Temperaturas del circuito de humos
- Rendimiento del generador de vapor

# Importo librerías y tablas de datos
-  [coolprop para calcular propiedades del agua y del vapor](http://www.coolprop.org/fluid_properties/IF97.html)
- tabla de propiedades de humos para calcular coeficiente de convección de humos
- tabla de entalpía de humos para calcular cp humos y balance de calor en cada paso

In [ ]:
!pip install CoolProp
import CoolProp.CoolProp as CP
import pandas as pd
import numpy as np
import math
url_propiedades_humos='https://raw.githubusercontent.com/AlejandroMuro/hello-world/master/Tabla%20propiedades%20humos.csv' #dirección donde está la tabla prop. de humos
url_entalpía_humos='https://raw.githubusercontent.com/AlejandroMuro/hello-world/master/entalp%C3%ADa%20humos01.csv' #dirección donde está la tabla entalpías de humos
url_entalpía_humos_carbón='https://raw.githubusercontent.com/AlejandroMuro/hello-world/master/Tablas%20h%20CARBON.csv' #dirección de la tabla de entalpías de humos de carbón
propiedades_humos = pd.read_csv(url_propiedades_humos,skiprows=[0,1],names=['Temperatura (°C)', 'Densidad (kg/m³)', 'k (kcal/h°Cm²) x10⁶', 'Viscocidad Cinematica (m²/s)', 'Pr'])
entalpía_humos=pd.read_csv(url_entalpía_humos,skiprows=[0,1],names=['Temperatura (°C)',	'h (kj/kg)',	'%CO2 ORSAT'])
entalpía_humos_carbón=pd.read_csv(url_entalpía_humos_carbón,skiprows=[0],names=['Temperatura (°C)',	'h (kj/kg)',	'%CO2 ORSAT'])
from scipy.optimize import fsolve

# Prueba de librería para cargar entalpía de Vapor y agua y Delta hfg

In [ ]:
#pruebo hallar entalpía de vapor,entalpía de agua y delta hfg con el coolprop!
#busco entalpía de vapor a 20 bar saturado
#H lo da en J/kg
t=450
a=CP.PropsSI('H','T',t,'Q',0.99,'IF97::Water') # q es calidad del vapor, que tan seco está el vapor
print(f'La entalpía del vapor saturado a {t}K es {a/1000} Kj/kg.') #coincide con el valor de tablas!

#lo puedo especificar con presión?
#vapor a 30 ata y 250°C
p2=30*101325 #presión en pascales
t2=250+273.15 #temperatura en kelvin
b=CP.PropsSI('H','P',p2,'T',t2,'IF97::Water') # q es calidad del vapor, que tan seco está el vapor
print(f'La entalpía del vapor a {p2} pascal y {t2} K es {b/1000} Kj/kg.') #coincide en valor con la que saqué de tablas!

#puedo sacar entalpía de agua?
t3=25+273.15 #agua a 25°C
p3=30*101325 #agua a 30 atmosferas
c=CP.PropsSI('H','P',p3,'T',t3,'IF97::Water')
print(f'La entalpía del agua a {t3} K y {p3} pascales es {c/1000} Kj/kg.') #coincide con la entalpía de agua sacada de tablas!

#puedo calcular hfg como una diferencia?
#quisiera calcular el hfg a 30 atmósferas
p4=30*101325
hvap=CP.PropsSI('H','P',p4,'Q',0.999,'IF97::Water') #lo calculo variando la calidad del vapor
hliq=CP.PropsSI('H','P',p4,'Q',0.001,'IF97::Water') #vapor de calidad 0.001 es agua
hfg=hvap-hliq
print(f'La entalpía de cambio de fase a {p4} pascales es {hfg/1000} Kj/kg.') #la entalpía de cambio de fase da lo mismo que en tablas!

#cual es la temperatura de saturación a 30 atmósferas?
t_sat=CP.PropsSI('T','P',p4,'Q',0.001,'IF97::Water')
print(f'La temperatura de saturación a {p4} pascales es {t_sat} Kelvin')

La entalpía del vapor saturado a 450K es 2754.1590205655352 Kj/kg.
La entalpía del vapor a 3039750 pascal y 523.15 K es 2854.51754815745 Kj/kg.
La entalpía del agua a 298.15 K y 3039750 pascales es 107.64738909497697 Kj/kg.
La entalpía de cambio de fase a 3039750 pascales es 1787.8717820387128 Kj/kg.
La temperatura de saturación a 3039750 pascales es 507.73954162718854 Kelvin


# Pruebo función de interpolación con la tabla Propiedades de humos

In [ ]:
print('Tabla de propiedades de humos:')
Temperatura=1200
#esta es la forma de llamar una fila de las propiedades de humos a una temperatura t
propiedades_humos[propiedades_humos['Temperatura (°C)']==Temperatura]

propiedades_humos[['Temperatura (°C)', 'Densidad (kg/m³)']]
#voy a precisar interpolar las propiedades según la temperatura de humos

#las columnas de propiedades de humos son: ['Temperatura (°C)', 'Densidad (kg/m³)', 'k (kcal/h°Cm²) x10⁶', 'Viscocidad Cinematica (m²/s)', 'Pr']
#las columnas de entalpia de humos son: ['Temperatura (°C)',	'h (kj/kg)',	'%CO2 ORSAT']

#para interpolar esta tabla hay que especificar temperatura y %CO2 en orsat!

def interpolo(tabla,propiedad, temperatura):
  '''tabla es el objeto tabla, propiedad es el nombre de la columna y temperatura la temperatura que vamos a interpolar'''
  x1=tabla['Temperatura (°C)']
  y1=tabla[propiedad]
  a=np.interp(temperatura,x1,y1)
  return temperatura,propiedad,a

b=interpolo(propiedades_humos,'Viscocidad Cinematica (m²/s)',33) #acordate, los valores son todos x 10^-7
c=interpolo(propiedades_humos,'Pr',1175)
print(b)
print(c)
entalpía_humos_carbón.head(598)

Tabla de propiedades de humos:
(33, 'Viscocidad Cinematica (m²/s)', 15.282)
(1175, 'Pr', 0.5625)


,Temperatura (°C),h (kj/kg),%CO2 ORSAT
0,0,0.0,8%
1,10,10.1,8%
2,20,20.2,8%
3,30,30.3,8%
4,40,40.5,8%
...,...,...,...
593,1910,2345.0,16%
594,1920,2358.5,16%
595,1930,2372.1,16%
596,1940,2385.7,16%


# Pruebo la función interpolación en la tabla entalpías de humos

In [ ]:
print('Entalpías Humos:')

entalpía_humos_carbón['Temperatura (°C)']
print(entalpía_humos_carbón.columns)
#explicito que quiero solo la tabla para %CO2 orsat de 6%
tabla6=entalpía_humos[entalpía_humos['%CO2 ORSAT']=='6%']
#interpolo en la tabla de %CO2 orsat=6% los datos son tabla, nombre de la columna de la tabla, temperatura
d=interpolo(tabla6 ,'h (kj/kg)',105)
print(d)
entalpía_humos_carbón[entalpía_humos_carbón['%CO2 ORSAT']=='12%'].head(201)
#print(entalpía_humos[entalpía_humos['Temperatura (°C)']==2000])

Entalpías Humos:
Index(['Temperatura (°C)', 'h (kj/kg)', '%CO2 ORSAT'], dtype='object')
(105, 'h (kj/kg)', 108.85)


,Temperatura (°C),h (kj/kg),%CO2 ORSAT
201,0,0.0,12%
202,10,10.1,12%
203,20,20.2,12%
204,30,30.3,12%
205,40,40.5,12%
...,...,...,...
397,1960,2384.0,12%
398,1970,2397.4,12%
399,1980,2410.8,12%
400,1990,2424.2,12%


#Calculo de Combustión
Debemos encontrar Ga y composición de humos

In [ ]:
#escribo el diccionario con los datos del fuel
c=0.85
h=0.08
o=0.05
n=0.02
s=0.000
h2o=0.226 #dato másico
fuel={'C':c,'H':h,'O':o,'N':n,'S':s,'H2O':h2o}
ash=0 #ceniza
tipo_fuel='No Carbón'#tipo de fuel

#los datos del fuel son porcentajes molares o másicos?
molar_másico='Másico'

#paso de base total a base seca para datos másicos
if molar_másico=='Másico':
  kg_bt=c+h+o+n+s+h2o # kg base total
  kg_bs=c+h+o+n+s #kg base seca
  bt_bs=kg_bs/kg_bt
elif molar_másico=='Molar':
  kg_bt=c*12+h*1+o*16+n*14+s*32+h2o*18
  kg_bs=c*12+h*1+o*16+n*14+s*32
  bt_bs=kg_bs/kg_bt


#escribo el diccionario con los datos de la combustión
#aquí defino las condiciones para la combustión del fuel
e=0.25 #exceso de aire
co2=0.137984 #porcentaje de co2 en humos
co=0 #porcentaje de co en humos
combustión={'exceso':e,'CO2':co2,'CO':co} #ordeno todas las variables en un diccionario

#escribo el diccionario con los datos de humos y aire
alfa=0
beta=0
nu=0
epsilon=0
delta=0
mu=0
gamma=0
humos={'Alfa':alfa,'Beta':beta,'Delta':delta,'Gamma':gamma,'Epsilon':epsilon,'Nu':nu,'Mu':mu}


#escribo cada una de las ecuaciones que hay que resolver
#sistema de la combustión química (probada, funciona!)
def sistema_alfaquimico(x):
  alfa=x[0]
  beta=x[1]
  delta=x[2]
  gamma=x[3]
  mu=x[4]
  y=[-fuel['C']/12+beta]#carbono
  y.append(-fuel['H']+2*gamma)#hidrógeno
  y.append(-2*alfa-fuel['O']/16+2*beta+2*delta+gamma)#oxígeno
  y.append(-fuel['N']/14-2*3.76*alfa+2*mu)#nitrogeno
  y.append(-fuel['S']/32+delta)#azufre
  return y

humos_comb_quimica={'Alfa':alfa,'Beta':beta,'Delta':delta,'Gamma':gamma,'Mu':mu} #combustión quimica son 5 incógnitas

def alfa_químico():
  comb_quimica=fsolve(sistema_alfaquimico,[1,1,1,1,1])#fsolve=algoritmo de resolución no lineal
  return comb_quimica[0]


#sistema con el fuel expresado en porcentaje másico
def sistema_másico(x):  
  alfa=x[0]
  beta=x[1]
  delta=x[2]
  gamma=x[3]
  epsilon=x[4]
  nu=x[5]
  mu=x[6]
  y=[-fuel['C']/12+beta+epsilon]#carbono
  y.append(-fuel['H']+2*gamma)#hidrógeno
  y.append(-2*alfa-fuel['O']/16+2*beta+epsilon+2*delta+2*nu+gamma)#oxígeno
  y.append(-fuel['N']/14-2*3.76*alfa+2*mu)#nitrogeno
  y.append(-fuel['S']/32+delta)#azufre

  if combustión['exceso']!=0:
    y.append((alfa-alfa_químico())/alfa_químico()-combustión['exceso'])#exceso
  if combustión['CO2']!=0:
    y.append(beta/(beta+delta+epsilon+mu+nu+gamma)-combustión['CO2'])#%co2
  if combustión['CO']!=0:
    y.append(epsilon/(beta+delta+epsilon+mu+nu+gamma)-combustión['CO'])#%co
  return y

#sistema con el fuel expresado en porcentaje molar
def sistema_molar(x):
  alfa=x[0]
  beta=x[1]
  delta=x[2]
  gamma=x[3]
  epsilon=x[4]
  nu=x[5]
  mu=x[6]
  y=[-fuel['C']+beta+epsilon]#carbono
  y.append(-fuel['H']+2*gamma)#hidrógeno
  y.append(-2*alfa-fuel['O']+2*beta+epsilon+2*delta+2*nu+gamma)#oxígeno
  y.append(-fuel['N']-2*3.76*alfa+2*mu)#nitrogeno
  y.append(-fuel['S']+delta)#azufre
  if combustión['exceso']!=0:
    y.append((alfa-alfa_químico())/alfa_químico()-combustión['exceso'])#exceso
  if combustión['CO2']!=0:
    y.append(beta/(beta+delta+epsilon+mu+nu+gamma)-combustión['CO2'])#%co2
  if combustión['CO']!=0:
    y.append(epsilon/(beta+delta+epsilon+mu+nu+gamma)-combustión['CO'])#%co
  return y

#planteo el sistema que hay que resolver
def sistema():
  if molar_másico=='Másico':
    a=fsolve(sistema_másico,[1,1,1,1,1,1,1])
    for i,j in zip(humos.keys(),a):
      humos[i]=j
  if molar_másico=='Molar':
    a=fsolve(sistema_molar,[1,1,1,1,1,1,1])
    for i,j in zip(humos.keys(),a):
      humos[i]=j


sistema()
Gasto_aire=138*humos['Alfa']
GH9H=Gasto_aire+1-ash-h2o #h2o es el agua que viene con el fuel
#prnt('GH9H',GH9H)
#porcentaje de co2 en humos ORSAT (%CO2 ORSAT)
pc_co2=(humos['Beta']+humos['Delta'])/(humos['Beta']+humos['Delta']+humos['Epsilon']+humos['Mu']+humos['Nu'])
#print(humos)
var_comb=pd.Series(humos, name='Moles/kg fuel')
print(f'Valores en base total')
print(var_comb)
print(f'Para pasar de base total a base seca divido entre {bt_bs:.2f}')
var_comb=var_comb/bt_bs
print(f'Valores en base seca')
print(var_comb)
print(f'Alfa químico: {alfa_químico():.2f}')
print(f'Gasto de aire: {Gasto_aire:.2f}') #el gasto de aire está bien!
#chequeé los resultados y están bien para combustible expresado en forma másica!
#hay un detalle, doy %CO=0 y me devuelve un valor de epsilon diferente de cero

#escribo composición de humos:
total=var_comb.sum()-var_comb['Alfa']
porcentajes=var_comb/total
print('Porcentajes en composición:')
print(porcentajes.rename({'Alfa':'No va','Beta':'CO2','Delta':'SO2','Gamma':'H2O','Epsilon':'CO','Nu':'O2','Mu':'N2'}))
#print(porcentajes)

Valores en base total
Alfa       1.115885e-01
Beta       7.600857e-02
Delta     -1.289998e-39
Gamma      4.000000e-02
Epsilon   -5.175239e-03
Nu         1.973009e-02
Mu         4.202872e-01
Name: Moles/kg fuel, dtype: float64
Para pasar de base total a base seca divido entre 0.82
Valores en base seca
Alfa       1.368076e-01
Beta       9.318651e-02
Delta     -1.581538e-39
Gamma      4.904000e-02
Epsilon   -6.344843e-03
Nu         2.418909e-02
Mu         5.152721e-01
Name: Moles/kg fuel, dtype: float64
Alfa químico: 0.09
Gasto de aire: 15.40
Porcentajes en composición:
No va    2.025750e-01
CO2      1.379840e-01
SO2     -2.341830e-39
H2O      7.261497e-02
CO      -9.394996e-03
O2       3.581749e-02
N2       7.629785e-01
Name: Moles/kg fuel, dtype: float64


# Funciones para el calculo de numeros importantes
Función reinolds, función nuselt forzado, cargo tablas de propiedades de vapor. Importo las clases fluido y geometría.

In [ ]:
#voy a definir una clase geometría
#radio interno de los tubos en metros
#El largo de los tubos en metros
#propiedades

#para calcular entalpías de humos interpolo de la tabla de entalpías de humos
#agregar que calcule el CO2 orsat y según eso elija el valor de la tabla adecuado entre 6%, 10%, 14% para fuel no carbón, 8%, 12%, 16% para fuel carbón
def h_humos(temperatura):
  '''Función que devuelve la entalpía de humos'''
  if tipo_fuel!='Carbón':
    h=interpolo(entalpía_humos[entalpía_humos['%CO2 ORSAT']=='10%'],'h (kj/kg)',temperatura)
  else:
    h=interpolo(entalpía_humos_carbón[entalpía_humos_carbón['%CO2 ORSAT']=='12%'],'h (kj/kg)',temperatura)
  return h[2]

class geometría:
  def __init__(self,radio_interno,largo,ntubos): #si no especifico cual es el numero de tubos, por defecto será 1
    '''Esta clase incluyo los datos de geometría, radios en metros, largo del intercambiador
    en metros y si el flujo es contracorriente. Incluyo funciones para calcular el area, la sección
    interna y la sección externa en metros cuadrados.'''
    self.ri=radio_interno
    self.l=largo
    self.nt=ntubos
    #area de intercambio
    self.a=0
    #sección interna
    self.si=0
    #sección externa
    self.q=0 #calor intercambiado
    self.ntu=0
    self.e=0
    self.calc_props()

  def area(self):
    '''superficie del cilindro, superficie de intercambio respecto al radio interno
    en m2'''
    self.a=2*math.pi*self.ri*self.l*self.nt

  def seccion_interna(self):
    '''superficie de sección de flujo interna en m2'''
    self.si=math.pi*self.ri**2
  
  def calc_props(self):
    '''calculo seccion interna y externa y el area de intercambio'''
    self.area()
    self.seccion_interna()



#voy a definir una clase fluido
#las velocidades de cada fluido en m/s
#las temperaturas de entrada de cada fluido en K

class fluido:
  def __init__(self,substancia,temperatura,tsalida):
    '''función que define los valores iniciales de los parámetros del objeto, estos son el tipo de sustancia, velocidad,
    temperatura inicial y cambio de estado si o no y flujo interno o no'''
    self.substancia=substancia
    self.v=0
    self.den=0
    #viscocidad cinematica
    self.vc=0
    #conductividad
    self.k=0
    #reinolds
    self.re=0
    self.pr=0
    self.gr=0
    self.nuselt=0
    self.h=0
    self.cmin=0
    self.t=temperatura
    self.ts=tsalida
    self.tm=temperatura
       
  def densidad(self):
    '''en kg/m3'''
    self.den=interpolo(propiedades_humos,'Densidad (kg/m³)',self.tm)[2]

  def visc_cinemat(self):
    '''en m2/s'''
    self.vc=interpolo(propiedades_humos,'Viscocidad Cinematica (m²/s)',self.tm)[2]*1e-6

  def conduct(self):
    '''en kW/k.m'''
    self.k=interpolo(propiedades_humos,'k (kcal/h°Cm²) x10⁶',self.tm)[2]*0.0011622 #multiplico por este numero para pasar a kj/sxcm2
      
  def prandlt_calc(self):
    self.pr=interpolo(propiedades_humos,'Pr',self.tm)[2]

  def tmedia(self):
    self.tm=(self.ts+self.t)/2
  
  def calc_cmin(self):
    self.cmin=GH9H*wf*(h_humos(self.t)-h_humos(self.ts))/(self.t-self.ts)/3600 #Kj/s
  
  def calc_props(self):
    '''función que calcula todas las propiedades a la temperatura media del fluido''' 
    self.tmedia()   
    self.densidad()
    self.visc_cinemat()
    self.conduct()
    self.prandlt_calc()
    self.calc_cmin()

#velocidad de humos
def v_humos(fluido,pase):
  fluido.v=GH9H*wf/(3600*interpolo(propiedades_humos,'Densidad (kg/m³)',fluido.tm)[2]*pase.si*pase.nt)
  return fluido.v

#función de reinolds
def reinolds(fluido,pase):
  fluido.re=(pase.ri)*2*fluido.v/fluido.vc
  a={'Regimen':'','Reinolds':0}
  if fluido.re<=2000:
    #print(f'El regimen es laminar Re={fluido.re:.2e}')
    a['Regimen']='laminar'
    a['Reinolds']=fluido.re       
  elif fluido.re>2000 and fluido.re<=10000:
    #print(f'El regimen es de transición Re={fluido.re:.2e}')
    a['Regimen']='transición'
    a['Reinolds']=fluido.re   
  elif fluido.re>10000:
    #print(f'El regimen es turbulento Re={fluido.re:.2e}')
    a['Regimen']='turbulento'
    a['Reinolds']=fluido.re

from copy import deepcopy

#función de prandlt
def prandlt(fluido,temp):
  fluido_pared=deepcopy(fluido)#uso un objeto fluido auxiliar
  fluido_pared.t=temp
  fluido_pared.calc_props()    
  return fluido_pared.pr

#función beta del agua para aproximar valores del beta (coeficiente de expansión térmico)
#luego lo voy a usar para hallar el nuselt en regimen laminar
#esta función no es valida, no tengo beta de humos!
def beta_agua(temperatura):
  '''Fución que calcula el gas'''
  beta=1/temperatura
  return beta

#función del Grashof
#no voy a poder calcular grashof de humos porque no tengo el beta de humos!
def grashof_calc(fluido,paso,T_pared):
  Gr = (9.81*beta_agua(fluido.t)*abs(T_pared-fluido.t)*(paso.ri*2)**3)/fluido.vc**2
  fluido.gr=Gr
  return Gr

#hay que modificar las funciones, ya no tengo la clase fluido con la que trabajé en el primer parcial
#función para el calculo del nuselt
#función de Nuselt (la saco de las tablas de convección forzada)
def nuselt(fluido,paso,T_pared):
  '''función que calcula el nuselt a partir de las props del fluido
  como el prandtl, el grashof y el reinolds y el prandtl wall para el caso exclusivo de un
  intercambiador de tubos concentricos'''
  grashof_calc(fluido,paso,T_pared)
  if fluido.re<=2000:
    fluido.nuselt=0.17*(fluido.re**0.33)*(fluido.pr**0.43)*(fluido.gr**0.1)*(fluido.pr/prandlt(fluido,T_pared))**0.25
    #print(f'El nuselt es: {fluido.nuselt}')
  elif fluido.re>2000 and fluido.re<=10000:
    #ko=0
    #reinolds=0
    #en este diccionario voy a poner los valores de ko para cada valor de reinolds
    #de la forma reinolds:ko
    dic_ko={2.2*10**3:2.2,2.3*10**3:3.6,2.5*10**3:4.9,3*10**3:7.5,3.5*10**3:10,4*10**3:12.2,5*10**3:16.5,6*10**3:20\
            ,7*10**3:24,8*10**3:27,9*10**3:30,10*10**3:33}
    x=list(dic_ko.keys())
    y=list(dic_ko.values())
    interpolation=np.poly1d(np.polyfit(x,y,11)) #interpolo con los valores del diccionario en un polinomio de grado 11 (mejor aproximación)
    ko=interpolation(fluido.re)

    fluido.nuselt=ko*(fluido.pr**0.43)*(fluido.pr/prandlt(fluido,T_pared))**0.25
    #print(f'El nuselt es: {fluido.nuselt}')
  elif fluido.re>10000:
    fluido.nuselt=0.021*(fluido.re**0.8)*(fluido.pr**0.43)*(fluido.pr/prandlt(fluido,T_pared))**0.25
    #print(f'El nuselt es: {fluido.nuselt}')
  return fluido.nuselt

#hago la función que calcula el h convección del intercambio del fuido
def calc_hconv(fluido,pase):
  hconv=(fluido.nuselt*fluido.k)/2*pase.ri
  fluido.h=hconv
  return hconv

def deltahfg(presión):
  '''Esta función devuelve el delta hfg para cierta presión en pascales'''
  hvap=CP.PropsSI('H','P',presión,'Q',0.999,'IF97::Water')/1000 #lo calculo variando la calidad del vapor
  hliq=CP.PropsSI('H','P',presión,'Q',0.001,'IF97::Water')/1000
  return hvap-hliq

#pruebo el calculo de un h conocido



# Planteo de los balances y ecuaciones de transferencia para resolver la caldera
Planteo:
- balance de calor de caldera
- balance de calor en el hogar
- ecuación de transferencia en el hogar
- calculo de coeficiente de convección en un paso
- calculo de q intercambiado por método NTU
- balance de calor en un paso


# Planteo loop para el cálculo de la caldera
El calculo de la caldera tiene una primera parte de entrada de datos del problema, Qps, Presión de operación de la caldera, kg de vapor producidos por hora, temperatura de entrada del agua, temperatura de referencia...
Tiene una segunda parte de definición de variables que serán los objetos humos y hogar o pases.
La tercera pase es el loop que tiene varias partes internamente.
La primera parte dentro del loop son las ecuaciones como el balance general de caldera y el loop de calculo de temperatura y calor de hogar.
La segunda parte es el loop de calculo de calor y temperatura de salida de los pases usando propiedades de humos y ntu trabajando cada pase como un intercambiador. La última parte compara la temperatura encontrada al final del último pase con la temperatura de chimenea supuesta en el comienzo. Si la temperatura se aproxima en un intervalo de 0.5 grados entonces el loop cierra. De lo contrario recalcula todo desde el principio utilizando la nueva temperatura de chimenea.

In [ ]:
#datos

def prnt(name,value):
  print(f'El valor de {name} es {value}')

#QPS en Kj/Kg
QPS=43890 #kj/kg
#gasto de vapor por hora
Gv_t=7000 #en kg vapor/hora
#temperatura de referencia
to=25
#supongo una t chimenea
t_chimenea=250
#supongo una temperatura de hogar
t_hogar=800
#cargo la presión de trabajo de la caldera
p_trabajo=10*101325 #en pascales
cp_vapor=0.5*4.184 #unidades: kj/kg vap
t_saturación=CP.PropsSI('T','P',p_trabajo,'Q',0.001,'IF97::Water')-273.15
prnt('t_saturación',t_saturación)
h2=CP.PropsSI('H','P',p_trabajo,'Q',1,'IF97::Water')/1000 #entalpía (kj/kg) de vapor calidad 1 a la presión de trabajo
#temperatura de agua de entrada
t1=65 #temperatura entrada del agua para pasar las temperaturas a kelvin sumo 273.15
h1=CP.PropsSI('H','P',p_trabajo,'T',t1+273.15,'IF97::Water')/1000 #entalpía (kj/kg) de agua a presión de trabajo a temperatura de entrada
#defino la variable global wf
wf=0


prnt('h1',h1)
prnt('h2',h2)
prnt('GH9H',GH9H)
prnt('Deltahfg',deltahfg(101325))
#balances que van a usar las variables globles de los datos:
#balance de calor general de la caldera recibe:
#QPS, h2, h1, GH9H, h (cantidad de hidógeno en el fuel), deltaHfg, to
#idea: si para cada balance sustituyo los valores que se y despejo la incógnita, puedo resolverlos usando fsolve
#voy a escribir todos los balances en el loop
#para el método NTU mis insumos son Ug, Chumos, area, Te de humos, T_saturación
#lo que obtengo es Q intercambiado
#luego hallo Ts_humos al plantear el balance de calor en el pase/hogar


#defino el fluido y los pasos por tubos
hogar=geometría(0.3,5,1) #defino hogar, rinterno, largo y n tubos
hogar.calc_props()
humo_hogar=fluido('humos',t_hogar,0) #humos a la temperatura de hogar sustancia, temperatura, t salida
humo_hogar.tmedia() #calculo tmedia
humo_hogar.calc_props() #calculo las propiedades a t media
v_humos(humo_hogar,hogar) #carga automaticamente la velocidad al fluido
#humo_chimenea=fluido('humos',0,t_chimenea)
#humo_chimenea.tmedia()
#humo_chimenea.calc_props()

n_pases=2 #si son la cantidad de pases sin incluir el hogar, entonces tomo 3 pases y 1 hogar, sinó serían 2 pases y 1 hogar
#escribo bajo el supuesto que el radio, el largo y el numero de caños es igual para todos los pases
radio_pases=0.025
largo_pases=5 #sin contar el hogar
n_caños=80
pase=[]
humo=[]
for i in range(n_pases):
  pase.append(geometría(radio_pases,largo_pases,n_caños))
  pase[i].calc_props()
  humo.append(fluido('humos',0,0))
  print(i)


#Panteo en orden los pasos del loop:
i=0
while True:
  #Resuelvo el balance de calor de la caldera con la tabla de entalpías de humos y obtengo Gv (kg vap/ kg fuel) 
  bgc=lambda x:-QPS*0.98+x*(h2-h1)+GH9H*(h_humos(t_chimenea)-h_humos(to))+9*fuel['H']*deltahfg(101325)+fuel['H2O']*(deltahfg(101325)+cp_vapor*(t_chimenea-to))
  Gv=float(fsolve(bgc,10))
  prnt('Gv',Gv)
  #Usando Gv_t calculo wf
  wf=float(Gv_t/Gv)
  prnt('wf',wf)
  #Con wf y planteando un loop, uso BQH y TQH y la tabla de entalpía de humos y despejo Qh, Th y h(Th)
  while True:
    prnt('t_hogar',t_hogar)
    bh=lambda x:-QPS*0.98+x+GH9H*(h_humos(t_hogar)-h_humos(to))+9*fuel['H']*deltahfg(101325)+fuel['H2O']*(deltahfg(101325)+cp_vapor*(t_hogar-to))
    hogar.q=float(fsolve(bh,10))    
    prnt('hogar.q',hogar.q)
    eth=lambda x:-hogar.q+(4.9*hogar.a*0.85/wf)*(((x+273.15)/100)**4-((150+t_sat+273.15)/100)**4)*(4.184) #multiplico por 4.184 para obtener Kj/kg fuel
    humo_hogar.t=float(fsolve(eth,100))
    prnt('h_humos(humo_hogar.t)',h_humos(humo_hogar.t))
    prnt('humo_hogar.t',humo_hogar.t)
    if abs(t_hogar-humo_hogar.t)<0.5:
      print('Aca pare!')
      break
    else:
      t_hogar=humo_hogar.t
  #defino las temperaturas de entrada y salida del primer paso para poder calcular las propiedades del humo
  humo[0].t=humo_hogar.t
  humo[0].ts=humo[0].t-200
  humo[0].tmedia()
  humo[0].calc_props()
  while True:
          
    t_salida=humo[i].ts #variable auxiliar que retiene un valor supuesto de t_salida del pase
    prnt(f'humo[{i}].t',humo[i].t)
    #prnt(f'humo[{i}].cmin',humo[i].cmin)
    humo[i].v=GH9H*wf/(3600*interpolo(propiedades_humos,'Densidad (kg/m³)',humo[i].tm)[2]*pase[i].si*pase[i].nt)
    prnt(f'humo[{i}].v',humo[i].v)
    #calculo Re, Nu, hconv, Ug de humos
    humo[i].re=(pase[i].ri)*2*humo[i].v/humo[i].vc
    prnt(f'humo[{i}].re',humo[i].re)
    nuselt(humo[i],pase[i],50+t_saturación)
    prnt(f'humo[{i}].nuselt',humo[i].nuselt)
    humo[i].h=(humo[i].nuselt*humo[i].k)/(2*pase[i].ri)    
    prnt(f'humo[{i}].h',humo[i].h)  
    prnt(f'pase[{i}].a',pase[i].a)
    prnt(f'humo[{i}].cmin',humo[i].cmin)  
      
    prnt(f'humo[{i}].k',humo[i].k)
    prnt(f'pase[{i}].ri',pase[i].ri)
    pase[i].ntu=humo[i].h*pase[i].a/humo[i].cmin
    
    pase[i].e=1-math.exp(-pase[i].ntu)
    prnt(f'pase[{i}].ntu',pase[i].ntu)
    prnt(f'pase[{i}].e',pase[i].e)

    #calculo el calor que se intercambia en el pase por ntu
    pase[i].q=pase[i].e*humo[i].cmin*(humo[i].t-t_saturación)
    #calculo el valor de humo.ts
    eccmin=lambda x: -pase[i].q+humo[i].cmin*(humo[i].t-x)+(wf*fuel['H2O']*cp_vapor*(humo[i].t-x)/3600) 
    humo[i].ts=float(fsolve(eccmin,400))
    ##bpc= lambda x:-x+GH9H*wf*(h_humos(humo[i].t)-h_humos(humo[i].ts))+wf*fuel['H2O']*(cp_vapor*(humo[i].t-humo[i].ts))
    ##pase[i].q=float(fsolve(bpc,100)) #kj/kg fuel
    prnt(f'pase[{i}].q',pase[i].q)
    ##edtml=lambda x:-pase[i].q+3600*humo[i].h*pase[i].a*((humo[i].t-t_saturación)-(x-t_saturación))/math.log(abs((humo[i].t-t_saturación)/(x-t_saturación)))
    ##humo[i].ts=float(fsolve(edtml,100)) #el factor de correción de dtml es 1 si la diferencia de temperatura entre uno de los fluidos y el otro es grande
    prnt(f'humo[{i}].ts',humo[i].ts) #en este caso se da que la diferencia de temperaturas es grande
    
    
    if abs(humo[i].ts-t_salida)<0.5 and i!=(n_pases-1):
      humo[i+1].t=humo[i].ts
      humo[i+1].ts=humo[i].t-200
      humo[i+1].tmedia()
      humo[i+1].calc_props()
      i+=1
      print('Siguiente pase!')
    elif abs(humo[i].ts-t_salida)<0.5 and i==(n_pases-1):
      print('Se terminaron los pases!')
      break      
    else:
      t_salida=humo[i].ts

  if abs(humo[n_pases-1].ts-t_chimenea)<0.5:
    print(f'La temperatura de chimenea calculada es  {humo[n_pases-1].ts:.2f}')
    print(f'La temperatura de chimenea supuesta es {t_chimenea}')
    print('La iteración se resolvió! Felicitaciones!')
    break
  else:
    print('Requiere otra iteración!')
    print(f'La temperatura de chimenea calculada es  {humo[n_pases-1].ts:.2f}')
    print(f'La temperatura de chimenea supuesta es {t_chimenea}')
    t_chimenea=humo[n_pases-1].ts
    i=0

#calculo la eficiencia:
eficiencia=Gv*(h2-h1)/QPS
prnt('eficiencia',eficiencia)


    
    



#comienzo un loop para el primer pase:

#calculo Cmin
#Calculo Area intercambio
#calculo NTU y obtengo epsilon
#calculo Qmax=Cmin*(Tentrada humos - T saturación)
#Calculo epsilon*Qmax= Q primer paso
#Calculo (con fsolve) la nueva temperatura de salida Ts1p'
#comparo con la supuesta y vuelvo a iterar si es necesario (diferencia < 0.5)


El valor de t_saturación es 180.45843535988
El valor de h1 es 272.8980312691355
El valor de h2 es 2777.6181244758395
El valor de GH9H es 16.173218750000004
El valor de Deltahfg es 2252.0276667412822
0
1
El valor de Gv es 14.725847928496995
El valor de wf es 475.3546304422866
El valor de t_hogar es 800
El valor de hogar.q es 26172.957639762742
El valor de h_humos(humo_hogar.t) es 1726.7589242558759
El valor de humo_hogar.t es 1425.5251290784265
El valor de t_hogar es 1425.5251290784265
El valor de hogar.q es 12714.495949305528
El valor de h_humos(humo_hogar.t) es 1397.3830654418364
El valor de humo_hogar.t es 1177.6969965204858
El valor de t_hogar es 1177.6969965204858
El valor de hogar.q es 18158.734923322438
El valor de h_humos(humo_hogar.t) es 1548.8414533425223
El valor de humo_hogar.t es 1292.3619949943777
El valor de t_hogar es 1292.3619949943777
El valor de hogar.q es 15654.952590323825
El valor de h_humos(humo_hogar.t) es 1483.2537241112389
El valor de humo_hogar.t es 1242.84373

/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


El valor de hogar.q es 16257.594596702082
El valor de h_humos(humo_hogar.t) es 1499.5734166680495
El valor de humo_hogar.t es 1255.2071338394314
El valor de t_hogar es 1255.2071338394314
El valor de hogar.q es 16469.34184595826
El valor de h_humos(humo_hogar.t) es 1505.214674097588
El valor de humo_hogar.t es 1259.480813710294
El valor de t_hogar es 1259.480813710294
El valor de hogar.q es 16376.083993871773
El valor de h_humos(humo_hogar.t) es 1502.7359690615174
El valor de humo_hogar.t es 1257.6030068647858
El valor de t_hogar es 1257.6030068647858
El valor de hogar.q es 16417.06044469097
El valor de h_humos(humo_hogar.t) es 1503.826207166679
El valor de humo_hogar.t es 1258.4289448232416
El valor de t_hogar es 1258.4289448232416
El valor de hogar.q es 16399.03728846735
El valor de h_humos(humo_hogar.t) es 1503.3468921932601
El valor de humo_hogar.t es 1258.0658274191364
Aca pare!
El valor de humo[0].t es 1258.0658274191364
El valor de humo[0].v es 54.95601431979995
El valor de humo[

# Planteo calculo de rendimiento de Caldera